In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pickle #used to store tokenizer value in binary format

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
# tokenization-a process that involves breaking down a piece of text into individual words or tokens.

from tensorflow.keras.preprocessing.sequence import pad_sequences
'''
This function is utilized to pad sequences, ensuring that all sequences have the same length.
In the context of natural language processing (NLP) and sequence-based models,
this is often done to make input sequences uniform in length, which is a requirement for feeding
data into many machine learning models. The function adds padding (usually zeros) to the beginning or end of sequences as needed.
'''

from tensorflow.keras.models import Sequential
'''
Sequential is a linear stack of layers that allows you to build a model layer by layer in a step-by-step fashion.
'''

from tensorflow.keras.layers import Embedding,LSTM,Dense
'''
The Embedding layer in a neural network is like a translator for words. It takes each word and turns it into a unique set of numbers
creating a way for the computer to understand and work with words. This is especially useful in tasks like
understanding relationships between words and solving language-related problems in things like text analysis or translation.
'''

In [ ]:
#read text file
with open('IndiaUS.txt','r',encoding='utf-8') as myfile:
  mytext=myfile.read()

In [ ]:
print(mytext)

Following a lavish state visit by Indian Prime Minister Narendra Modi to Washington, US President Joe Biden has called his country's partnership with India among the "most consequential in the world". The BBC's Vikas Pandey and Soutik Biswas explore the factors that contribute to the visit's potential in strengthening the ties between the two nations.

The US's relationship with India - the world's most populous country - is "stronger, closer and more dynamic than any time in history", Mr Biden said at the completion of a pomp-filled state visit by Mr Modi to the White House.

The remark may not be an exaggeration. "This summit suggests that the relationship has been transformed. It underscores just how broad and deep it has become in a relatively short time," says Michael Kugelman of The Wilson Center, an American think-tank.

A key reason is that Washington is keen to draw India closer so that it can act as a counterbalance to China's growing influence in the Indo-Pacific. India-US t

In [ ]:
mytokenizer=Tokenizer()
# Tokenization is a common preprocessing step in machine learning models that work with text data.

In [ ]:
mytokenizer.fit_on_texts([mytext])
total_words=len(mytokenizer.word_index)+1  #we add one as out of vocabulary token
total_words

599

In [ ]:
mytokenizer.word_index

{'the': 1,
 'to': 2,
 'in': 3,
 'a': 4,
 'and': 5,
 'india': 6,
 'of': 7,
 'mr': 8,
 'us': 9,
 'is': 10,
 'modi': 11,
 'has': 12,
 'that': 13,
 'biden': 14,
 'as': 15,
 'with': 16,
 'not': 17,
 "india's": 18,
 'for': 19,
 'but': 20,
 'on': 21,
 'washington': 22,
 'an': 23,
 'it': 24,
 'says': 25,
 'will': 26,
 'are': 27,
 'indian': 28,
 'prime': 29,
 'minister': 30,
 'this': 31,
 'have': 32,
 'trade': 33,
 'visit': 34,
 'by': 35,
 'president': 36,
 'relationship': 37,
 'been': 38,
 'about': 39,
 'strategic': 40,
 'more': 41,
 'up': 42,
 'during': 43,
 'also': 44,
 'make': 45,
 'sirohi': 46,
 'china': 47,
 'air': 48,
 'force': 49,
 'state': 50,
 'his': 51,
 'ties': 52,
 'two': 53,
 'at': 54,
 'white': 55,
 'house': 56,
 'be': 57,
 'lot': 58,
 'ms': 59,
 'technology': 60,
 'jet': 61,
 'semiconductor': 62,
 'now': 63,
 'first': 64,
 'was': 65,
 'global': 66,
 'one': 67,
 'he': 68,
 'without': 69,
 'they': 70,
 'narendra': 71,
 'joe': 72,
 'world': 73,
 'potential': 74,
 'how': 75,
 'becom

In [ ]:
#save as pickle format
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(mytokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
!cp '/content/tokenizer.pickle'  '/content/drive/MyDrive/RNN_LSTM'
!cp '/content/IndiaUS.txt' '/content/drive/MyDrive/RNN_LSTM'

In [ ]:
my_input_sequences = []
for line in mytext.split('\n'):
    token_list = mytokenizer.texts_to_sequences([line])[0]
    # print(token_list)
    for i in range(1,len(token_list)):
      #n_gram sequence
      my_ngram_sequence=token_list[:i+1]
      my_input_sequences.append(my_ngram_sequence)



In [ ]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
max_sequence_len

83

In [ ]:
'''
padding 2 types post padding and pre padding
padding used to make all sentences in same length
we got above max sequence len is 83 so we going to make all sentences as the length of max_sequence_len by adding 0 in the empty spaces
'''
input_sequence=np.array(pad_sequences(my_input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequence

array([[  0,   0,   0, ...,   0,  99,   4],
       [  0,   0,   0, ...,  99,   4, 177],
       [  0,   0,   0, ...,   4, 177,  50],
       ...,
       [  0,   0,   0, ..., 176, 598,  25],
       [  0,   0,   0, ..., 598,  25,  59],
       [  0,   0,   0, ...,  25,  59,  46]], dtype=int32)

In [ ]:
x=input_sequence[:,:-1]
y=input_sequence[:,-1]

In [ ]:
x

array([[  0,   0,   0, ...,   0,   0,  99],
       [  0,   0,   0, ...,   0,  99,   4],
       [  0,   0,   0, ...,  99,   4, 177],
       ...,
       [  0,   0,   0, ...,   5, 176, 598],
       [  0,   0,   0, ..., 176, 598,  25],
       [  0,   0,   0, ..., 598,  25,  59]], dtype=int32)

In [ ]:
y

array([  4, 177,  50, ...,  25,  59,  46], dtype=int32)

In [ ]:
# y convert to binary vectors using one hot encoding
y=np.array(tf.keras.utils.to_categorical(y,num_classes=total_words))

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
'''
ARCHITECHTURE BUILDING
'''
model=Sequential()

'''
1.Embedding layer
'''
#input_dim=total no of unique words,size of input(higher dimensionality space)
#output_dim=lower dimensionality  that we need to map
#input_length=max sequence length excluding y

                                          #output_dim is a hyperparameter ,we can change it based on the performance
model.add(Embedding(input_dim=total_words,output_dim=100,input_length=max_sequence_len-1)) #embedding layer

'''
2.LSTM layer
'''
model.add(LSTM(150)) #no of nodes in lstm is  a hyperparameter we can increase or decrease this as per the choice
                    #lstm hidden layer must be greater than output dim
'''
3.Dense layer
'''
model.add(Dense(total_words,activation='softmax'))


In [ ]:
'''
COMPILING
'''
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x,y,epochs=100,verbose=1)

Epoch 1/100
43/43 [==============================] - 15s 164ms/step - loss: 6.1761 - accuracy: 0.0489
Epoch 2/100
43/43 [==============================] - 5s 115ms/step - loss: 5.8095 - accuracy: 0.0576
Epoch 3/100
43/43 [==============================] - 4s 106ms/step - loss: 5.7244 - accuracy: 0.0576
Epoch 4/100
43/43 [==============================] - 2s 54ms/step - loss: 5.6692 - accuracy: 0.0576
Epoch 5/100
43/43 [==============================] - 2s 37ms/step - loss: 5.5798 - accuracy: 0.0576
Epoch 6/100
43/43 [==============================] - 1s 35ms/step - loss: 5.4699 - accuracy: 0.0591
Epoch 7/100
43/43 [==============================] - 1s 35ms/step - loss: 5.3335 - accuracy: 0.0737
Epoch 8/100
43/43 [==============================] - 1s 22ms/step - loss: 5.1531 - accuracy: 0.0802
Epoch 9/100
43/43 [==============================] - 1s 17ms/step - loss: 4.9281 - accuracy: 0.1094
Epoch 10/100
43/43 [==============================] - 1s 11ms/step - loss: 4.6989 - accuracy: 0.

In [ ]:
model.save('/content/drive/MyDrive/RNN_LSTM/word_prediction.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


prediction

In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

In [ ]:
#loading tokenizer file from trained data
with open('/content/tokenizer.pickle','rb') as handle:
  mytokenizer=pickle.load(handle)

model=tf.keras.models.load_model('/content/drive/MyDrive/RNN_LSTM/word_prediction.h5')

In [ ]:
model.input_shape[1]

82

In [ ]:
input_text='modi'
predict_next_words=5

for i in range(predict_next_words):
  token_list=mytokenizer.texts_to_sequences([input_text])[0]
  token_list=pad_sequences([token_list],maxlen=model.input_shape[1],padding='pre')
  predicted=model.predict(token_list)
  predicted_index=np.argmax(predicted,axis=1)
  output_word=mytokenizer.index_word[predicted_index[0]]
  input_text += " " + output_word
print(input_text)


1/1 [==============================] - 0s 18ms/step
modi is the whole there is
